# `libiio`: library for interfacing with IIO devices

XADC is currently the only IIO device on Red Pitaya.
XADC provides measurements of Zynq chip temperature, supply voltages and a set of analog inputs.
The set of measured analog inputs is defined by a device tree node.

The kernel IIO driver provides a `sysfs` interface for accessing XADC measurements.
There are actually two drivers running at the same time.

## `compatible = "xlnx,zynq-xadc-1.00.a";`

This driver is accessing XADC over JTAG, and provides only
temperature and supply voltage measurements, at a slow data rate.
The advantage is, it does not require FPGA code and is therefore always available.
All data is accessed over `sysfs`.

## `compatible = "xlnx,axi-xadc-1.00.a";`

This driver requires FPGA code connecting the XADC to the CPU over the AXI bus.
The full feature set is provided.
External voltage measurements are available with the full data rate.
In addition to the `sysfs` interface a cyclic buffer can be obtained from a device node.

In [ ]:
# load 'classic' FPGA bitstream, it contains an AXI connected XADC and interrupt
#import os
#os.system('cat /opt/redpitaya/fpga/classic/fpga.bit > /dev/xdevcfg')

The next code is using Python bindings to `libiio` list the functionality exposed over `sysfs`.

In [ ]:
#!/usr/bin/env python
#
# Copyright (C) 2015 Analog Devices, Inc.
# Author: Paul Cercueil <paul.cercueil@analog.com>
#
# This library is free software; you can redistribute it and/or
# modify it under the terms of the GNU Lesser General Public
# License as published by the Free Software Foundation; either
# version 2.1 of the License, or (at your option) any later version.
#
# This library is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU
# Lesser General Public License for more details.

import iio
from sys import argv

def main():
	print('Library version: %u.%u (git tag: %s)' % iio.version)

	if len(argv) == 3 and argv[1] == '--uri':
		uri = argv[2]
	else:
		contexts = iio.scan_contexts()
		if len(contexts) > 1:
			print('Multiple contexts found. Please select one using --uri:')
			for index, each in enumerate(contexts):
				print('\t%d: %s [%s]' % (index, contexts[each], each))
			return

		uri = next(iter(contexts), None)

	ctx = iio.Context(uri)

	if uri is not None:
		print('Using auto-detected IIO context at URI \"%s\"' % uri)

	print('IIO context created: ' + ctx.name)
	print('Backend version: %u.%u (git tag: %s)' % ctx.version)
	print('Backend description string: ' + ctx.description)

	if len(ctx.attrs) > 0:
		print('IIO context has %u attributes:' % len(ctx.attrs))
	for attr, value in ctx.attrs.items():
		print('\t' + attr + ': ' + value)

	print('IIO context has %u devices:' % len(ctx.devices))

	for dev in ctx.devices:
		print('\t' + dev.id + ': ' + dev.name)

		if dev is iio.Trigger:
			print('Found trigger! Rate: %u Hz' % dev.frequency)

		print('\t\t%u channels found:' % len(dev.channels))

		for chn in dev.channels:
			print('\t\t\t%s: %s (%s)' % (chn.id, chn.name or "", 'output' if chn.output else 'input'))

			if len(chn.attrs) != 0:
				print('\t\t\t%u channel-specific attributes found:' % len(chn.attrs))

			for attr in chn.attrs:
				try:
					print('\t\t\t\t' + attr + ', value: ' + chn.attrs[attr].value)
				except OSError as e:
					print('Unable to read ' + attr + ': ' + e.strerror)

		if len(dev.attrs) != 0:
			print('\t\t%u device-specific attributes found:' % len(dev.attrs))

		for attr in dev.attrs:
			try:
				print('\t\t\t' + attr + ', value: ' + dev.attrs[attr].value)
			except OSError as e:
				print('Unable to read ' + attr + ': ' + e.strerror)

		if len(dev.debug_attrs) != 0:
			print('\t\t%u debug attributes found:' % len(dev.debug_attrs))

		for attr in dev.debug_attrs:
			try:
				print('\t\t\t' + attr + ', value: ' + dev.debug_attrs[attr].value)
			except OSError as e:
				print('Unable to read ' + attr + ': ' + e.strerror)

if __name__ == '__main__':
	main()

The next example is reading measurements provided by the `zynq-xadc` driver as `sysfs` files.
Based on my experience an undocumented ratio of 1000 has to be used to achieve correct results.

Values `raw`, `offset` and `scale` are used to calculate the temperature.

$$ T = (raw + offset) \cdot scale $$

Values `raw` and `scale` are used to calculate supply voltages.

$$ U = raw \cdot scale $$

In [ ]:
import iio

ctx = iio.Context()
dev = ctx.devices[2]

for chn in dev.channels:
    print ('{}: {} ({})'.format(chn.id, chn.name if chn.name else "", 'output' if chn.output else 'input'))
    raw    = chn.attrs['raw'].value
    scale  = chn.attrs['scale'].value
    if len(chn.attrs) == 3:
        offset = chn.attrs['offset'].value
        print ('\t({}+{})*{}/1000={}'.format(raw, offset, scale, (int(raw)+int(offset))*float(scale)/1000))
    else:
        print ('\t{}*{}/1000={}'.format(raw, scale, int(raw)*float(scale)/1000))

The next example is reading measurements provided by the `axi-xadc` driver as `sysfs` files.
Based on my experience an undocumented ratio of 1000 has to be used to achieve correct results.

The equations are the same as in the previous example.
This interface also provides slow analog input pin voltages `vaux0`, `vaux1`, `vaux8` and `vaux9`.

In [ ]:
import iio

ctx = iio.Context()
dev = ctx.devices[3]

for chn in dev.channels:
    print ('{}: {} ({})'.format(chn.id, chn.name if chn.name else "", 'output' if chn.output else 'input'))
    raw    = chn.attrs['raw'].value
    scale  = chn.attrs['scale'].value
    if len(chn.attrs) == 6:
        offset = chn.attrs['offset'].value
        print ('\t({}+{})*{}/1000={}'.format(raw, offset, scale, (int(raw)+int(offset))*float(scale)/1000))
    else:
        print ('\t{}*{}/1000={}'.format(raw, scale, int(raw)*float(scale)/1000))

The following code is an attempt to get a buffer containing XADC samples.
For now it crashes the Linux kernel, requiering a reboot.

I can get it to work in CLI using C example:
```
/home/jupyter/libiio/examples/dummy-iiostream -d xadc -t xadc0-samplerate -b 64 -c 2
```
But I have to comment out `zynq-xadc` in the device tree,
since both have the same device name (kernel driver name).

In [ ]:
import numpy as np

data_size = 64

import iio
ctx = iio.Context()
trg = ctx.devices[1]
dev = ctx.devices[3]
# due to a probable bug in the axi-xadc driver all channels have to be enabled,
# otherwise the driver crashes the kernel
for chn in dev.channels:
    chn.enabled = True
dev.trigger = trg
buf = iio.Buffer(dev, data_size, cyclic=False)

print ([dev.channels[i].id           for i in range(len(dev.channels))])
print ([dev.channels[i].scan_element for i in range(len(dev.channels))])
print ([dev.channels[i].enabled     for i in range(len(dev.channels))])

In [ ]:
cnt = buf.refill()

In [ ]:
data_buf = buf.read()
# data = np.recarray(data_size, data_type, buf=data_buf)
data = np.frombuffer(data_buf, np.dtype('uint16'))
# reshape array and transpose
data = data.reshape((data_size, len(dev.channels))).T

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.plotly as py
import plotly.graph_objs as go

init_notebook_mode(connected=True)

data_x = np.linspace(0, 1, data_size)

# temperature trace
chn = dev.channels[0]
scale  = chn.attrs['scale'].value
offset = chn.attrs['offset'].value
temp0 = go.Scatter(
    x = data_x,
    y = ((data[0]>>4) + int(offset)) * float(scale) / 1000,
    mode = 'lines',
    name = 'temp0'
)

# voltage traces
traces = []
for chn in dev.channels:
    if len(chn.attrs) == 5:
        scale  = chn.attrs['scale'].value
        traces.append(go.Scatter(
            x = data_x,
            y = (data[0]>>4) * float(scale) / 1000,
            mode = 'lines',
            name = chn.name
        ))

# Plot and embed in ipython notebook!
iplot([temp0], show_link=False)
iplot(traces, show_link=False)